# Pre-Processing using NLTK

In [ ]:
import nltk

In [2]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [5]:
from nltk.corpus import twitter_samples

In [6]:
import pandas as pd
import csv

all_tweets = pd.read_csv('/home/muskan_2117/Downloads/SemEval2017_train_data.csv')
all_tweets.shape

(50248, 4)

## Tokenization

In [25]:
from nltk.tokenize import word_tokenize

nltk_tokens = []
NaNCount = 0

for i in range(len(all_tweets)):
    if all_tweets.text[i] == 'Not Available':
        nltk_tokens.append(['NaN'])
        NaNCount = NaNCount + 1
    else:
        tmp = word_tokenize(str(all_tweets.text[i]))
        nltk_tokens.append(tmp) 

In [26]:
nltk_tokens

[['Won',
  'the',
  'match',
  '#',
  'getin',
  '.',
  'Plus',
  ',',
  'tomorrow',
  'is',
  'a',
  'very',
  'busy',
  'day',
  ',',
  'with',
  'Awareness',
  'Day',
  "'s",
  'and',
  'debates',
  '.',
  'Gulp',
  '.',
  'Debates',
  '...'],
 ['Some',
  'areas',
  'of',
  'New',
  'England',
  'could',
  'see',
  'the',
  'first',
  'flakes',
  'of',
  'the',
  'season',
  'Tuesday',
  '.'],
 ['NaN'],
 ['NaN'],
 ['NaN'],
 ['NaN'],
 ['@',
  'mariakaykay',
  'aga',
  'tayo',
  'tomorrow',
  'ah',
  '.',
  ':',
  ')',
  'Good',
  'night',
  ',',
  'Ces',
  '.',
  'Love',
  'you',
  '!',
  '&',
  'gt',
  ';',
  ':',
  'D',
  '&',
  'lt',
  ';'],
 ['Tina',
  'Fey',
  '&',
  'amp',
  ';',
  'Amy',
  'Poehler',
  'are',
  'hosting',
  'the',
  'Golden',
  'Globe',
  'awards',
  'on',
  'January',
  '13',
  '.',
  'What',
  'do',
  'you',
  'think',
  '?'],
 ['Lunch',
  'from',
  'my',
  'new',
  'Lil',
  'spot',
  '...',
  'THE',
  'COTTON',
  'BOWL',
  '...',
  '.pretty',
  'good',
  '#

In [27]:
all_tweets['tokens'] = nltk_tokens 

In [28]:
import numpy as np

data = np.array([np.arange(len(all_tweets) - NaNCount)]*2).T
tweets = pd.DataFrame(data, columns = ['label','tokens']) 

j = 0

for i in range(len(all_tweets)):
    if all_tweets.tokens[i] == ['NaN']:
        continue
    else:
        tweets['label'][j] = all_tweets['label'][i]
        tweets['tokens'][j] = all_tweets['tokens'][i]
        j = j + 1

/home/muskan_2117/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/muskan_2117/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [29]:
print('total tweets', len(all_tweets))
print('NaNCount', NaNCount)
print('available tweets', len(tweets))

total tweets 50248
NaNCount 8313
available tweets 41935


## Glove embeddings

In [30]:
import gensim

model = gensim.models.KeyedVectors.load_word2vec_format('/home/muskan_2117/DAIICT/Sem2/IT550/project_datasets/datastories-semeval2017-task4-master/embeddings/glove.txt', binary=False)

In [31]:
model_w2v = gensim.models.KeyedVectors.load_word2vec_format('/home/muskan_2117/DAIICT/Sem2/IT550/project_datasets/datastories-semeval2017-task4-master/embeddings/glove.txt', binary=False)

In [32]:
all_embeddings = []

for i in range(len(tweets)):
    token = tweets['tokens'][i]
    embeddings = []
    for j in range(len(token)):
        try:
            embeddings.append(model[token[j]])
        except KeyError:
            continue
    all_embeddings.append(embeddings)
            

In [33]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model_w2v[word].reshape((1, size))
            count += 1
        except KeyError:   # handling the case where the token is not in vocabulary
            continue
            
    if count != 0:
        vec /= count
    return vec

In [34]:
wordvec_arrays = np.zeros((len(tweets), 300))

for i in range(len(tweets)):
    wordvec_arrays[i,:] = word_vector(tweets['tokens'][i], 300)
    
wordvec_df = pd.DataFrame(wordvec_arrays)
wordvec_df.shape

(41935, 300)

In [35]:
len(all_embeddings)

41935

In [36]:
tweets['glove_embeddings'] = all_embeddings

## Train and test sets

In [37]:
from sklearn.model_selection import train_test_split

#X = tweets.glove_embeddings
X = tweets['glove_embeddings']
#X = X.reshape(-1, 1)
y = tweets.label
#y = np.array(tweets['label'])
#y = y.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(wordvec_df, y, test_size=0.11, random_state=42)

In [38]:
X_train.shape

(37322, 300)

In [39]:
X_test.shape

(4613, 300)

## SVM

In [40]:
from sklearn import svm

clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [41]:
from sklearn.metrics import classification_report, confusion_matrix

labels = ['positive', 'neutral', 'negative']

print(classification_report(y_test, y_pred, target_names=labels))

print(confusion_matrix(y_test, y_pred, labels))

              precision    recall  f1-score   support

    positive       0.59      0.28      0.37       697
     neutral       0.60      0.78      0.67      2097
    negative       0.70      0.60      0.65      1819

   micro avg       0.63      0.63      0.63      4613
   macro avg       0.63      0.55      0.57      4613
weighted avg       0.64      0.63      0.62      4613

[[1093  685   41]
 [ 372 1630   95]
 [  86  419  192]]


## Logistic Regression 

In [42]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(solver='sag')

classifier.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=labels))

print(confusion_matrix(y_test, y_pred, labels))

/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


              precision    recall  f1-score   support

    positive       0.59      0.28      0.37       697
     neutral       0.60      0.78      0.67      2097
    negative       0.70      0.60      0.65      1819

   micro avg       0.63      0.63      0.63      4613
   macro avg       0.63      0.55      0.57      4613
weighted avg       0.64      0.63      0.62      4613

[[1093  685   41]
 [ 372 1630   95]
 [  86  419  192]]


In [43]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(solver='newton-cg')

classifier.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=labels))

print(confusion_matrix(y_test, y_pred, labels))

/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


              precision    recall  f1-score   support

    positive       0.59      0.28      0.37       697
     neutral       0.60      0.78      0.67      2097
    negative       0.70      0.60      0.65      1819

   micro avg       0.63      0.63      0.63      4613
   macro avg       0.63      0.55      0.57      4613
weighted avg       0.64      0.63      0.62      4613

[[1093  685   41]
 [ 372 1630   95]
 [  86  419  192]]


In [44]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(solver='saga')

classifier.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=labels))

print(confusion_matrix(y_test, y_pred, labels))

/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


              precision    recall  f1-score   support

    positive       0.59      0.28      0.37       697
     neutral       0.60      0.78      0.67      2097
    negative       0.70      0.60      0.65      1819

   micro avg       0.63      0.63      0.63      4613
   macro avg       0.63      0.55      0.57      4613
weighted avg       0.64      0.63      0.62      4613

[[1093  685   41]
 [ 372 1630   95]
 [  86  419  192]]


In [45]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(solver='lbfgs')

classifier.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, target_names=labels))

print(confusion_matrix(y_test, y_pred, labels))

/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


              precision    recall  f1-score   support

    positive       0.59      0.28      0.37       697
     neutral       0.60      0.78      0.67      2097
    negative       0.70      0.60      0.65      1819

   micro avg       0.63      0.63      0.63      4613
   macro avg       0.63      0.55      0.57      4613
weighted avg       0.64      0.63      0.62      4613

[[1093  685   41]
 [ 372 1630   95]
 [  86  419  192]]


## SGD

In [62]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=1000)
clf.fit(X_train, y_train)

/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=1000,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [63]:
y_pred = clf.predict(X_test)

In [64]:
print(classification_report(y_test, y_pred, target_names=labels))

print(confusion_matrix(y_test, y_pred, labels))

              precision    recall  f1-score   support

    positive       0.70      0.14      0.23       697
     neutral       0.60      0.78      0.68      2097
    negative       0.66      0.64      0.65      1819

   micro avg       0.63      0.63      0.63      4613
   macro avg       0.65      0.52      0.52      4613
weighted avg       0.64      0.63      0.60      4613

[[1162  645   12]
 [ 439 1627   31]
 [ 148  451   98]]


In [65]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss="modified_huber", penalty="l2", max_iter=1000)
clf.fit(X_train, y_train)
        
y_pred = clf.predict(X_test)
        
print(classification_report(y_test, y_pred, target_names=labels))

print(confusion_matrix(y_test, y_pred, labels))       

/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

    positive       0.57      0.33      0.42       697
     neutral       0.61      0.76      0.68      2097
    negative       0.70      0.61      0.65      1819

   micro avg       0.63      0.63      0.63      4613
   macro avg       0.62      0.57      0.58      4613
weighted avg       0.64      0.63      0.63      4613

[[1101  665   53]
 [ 380 1596  121]
 [ 103  362  232]]


In [66]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss="log", penalty="l2", max_iter=1000)
clf.fit(X_train, y_train)
        
y_pred = clf.predict(X_test)
        
print(classification_report(y_test, y_pred, target_names=labels))

print(confusion_matrix(y_test, y_pred, labels))       

/home/muskan_2117/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

    positive       0.60      0.29      0.39       697
     neutral       0.60      0.77      0.68      2097
    negative       0.69      0.61      0.65      1819

   micro avg       0.63      0.63      0.63      4613
   macro avg       0.63      0.56      0.57      4613
weighted avg       0.64      0.63      0.62      4613

[[1113  666   40]
 [ 391 1611   95]
 [ 105  391  201]]
